**If you are interested in this notebook, please check [EDA about Pressure with Colored Charts](https://www.kaggle.com/marutama/eda-about-pressure-with-colored-charts) out as well.It will be the latest and most beautiful update.**

**It seems that Part 2 is not often seen, so I will add a description. Although it is "other than the main mode" that does not show 8% of the total, it is an interesting mode that can be classified very clearly.**

This is Part 2 of the series notebook, Pressure non-main mode EDA. Part 1 is [here](https://www.kaggle.com/marutama/eda-about-pressure-part-1), dealing with the main mo.

It's long, so I'll write an overview.
- I made the conditions based on the chart shape of u_in and puressure with the feeling that I became AI. I hope it will be a hint for Feature engineering.
- Feature addition has been sped up by avoiding groupby as much as possible.
- I also made the vibration coefficient of u_in. Take the diff of u_in and count how many times the sign of the diff is inverted. It's quite convenient.
- Part 2 is specialized for other than main mode. It accounts for 8% of the total.
- The u_in, pressure graph has a layered pattern more clearly than in the main mode.
- Since R_C in non-main mode is only 50_10, classification by R_C cannot be used.
- As a layered feature point of Pressure, the Pressure value when u_out becomes 1 is referred to. If you make a histogram, multiple peaks will appear neatly.
- Strangely, you can clearly distinguish between those with a time_step end time of less than 2.65 seconds and those with a longer time_step. The graph shape is significantly different.
- In the pattern of time_end> 2.65, it seems that it is possible to predict the number of layers by clearly using the mean value of u_in.

See also [Part 1](https://www.kaggle.com/marutama/eda-about-pressure-part-1)!

I think there are many places where the explanation is insufficient. Please comment if you request.

This notebook is a continuation of:
- [EDA about time_step and u_out](https://www.kaggle.com/marutama/eda-about-time-step-and-u-out).
- [EDA about u_in](https://www.kaggle.com/marutama/eda-about-u-in)

If you find it useful, please upvote it as well.。

Chart Plot referred to [Ventilator Pressure Prediction: EDA, FE and models](https://www.kaggle.com/artgor/ventilator-pressure-prediction-eda-fe-and-models). 

For the R_C distribution part, I referred to [Ventilator Pressure simple EDA](https://www.kaggle.com/currypurin/ventilator-pressure-simple-eda).

Thank you very much.

The importance of the features introduced in the "EDA about" series below:
- [EDA about: LSTM Feature Importance](https://www.kaggle.com/marutama/eda-about-lstm-feature-importance)

And [finetune of Tensorflow Bi-LSTM EDA about](https://www.kaggle.com/marutama/finetune-of-tensorflow-bi-lstm-eda-about) is for Modeling.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load CSV

In [ ]:
oj = os.path.join
path = '../input/ventilator-pressure-prediction'
train = pd.read_csv(oj(path, 'train.csv'))
test  = pd.read_csv(oj(path, 'test.csv'))
sub   = pd.read_csv(oj(path, 'sample_submission.csv'))

# Add features

Groupby is slow, so I don't use it as much as possible.

In [ ]:
bid_list = list(train['breath_id'].unique())

In [ ]:
%%time
train['R_C'] = [f'{r:02}_{c:02}' for r, c in zip(train['R'], train['C'])]
RCorder = list(np.sort(train['R_C'].unique()))
#RCorder

In [ ]:
%%time
# fast area calculation
train['time_delta'] = train['time_step'].diff()
train['time_delta'].fillna(0, inplace=True)
train['time_delta'].mask(train['time_delta'] < 0, 0, inplace=True)
train['tmp'] = train['time_delta'] * train['u_in']
train['area'] = train.groupby('breath_id')['tmp'].cumsum()

In [ ]:
%%time
# u_in: max, min, mean, std 
u_in_max_dict = train.groupby('breath_id')['u_in'].max().to_dict()
train['u_in_max'] = train['breath_id'].map(u_in_max_dict)
u_in_min_dict = train.groupby('breath_id')['u_in'].min().to_dict()
train['u_in_min'] = train['breath_id'].map(u_in_min_dict)
u_in_mean_dict = train.groupby('breath_id')['u_in'].mean().to_dict()
train['u_in_mean'] = train['breath_id'].map(u_in_mean_dict)
u_in_std_dict = train.groupby('breath_id')['u_in'].std().to_dict()
train['u_in_std'] = train['breath_id'].map(u_in_std_dict)

In [ ]:
# u_in_half is time:0 - time point of u_out:1 rise (almost 1.0s)
train['tmp'] = train['u_out']*(-1)+1 # inversion of u_out
train['u_in_half'] = train['tmp'] * train['u_in']

In [ ]:
%%time
# u_in_half: max, min, mean, std
u_in_half_max_dict = train.groupby('breath_id')['u_in_half'].max().to_dict()
train['u_in_half_max'] = train['breath_id'].map(u_in_half_max_dict)
u_in_half_min_dict = train.groupby('breath_id')['u_in_half'].min().to_dict()
train['u_in_half_min'] = train['breath_id'].map(u_in_half_min_dict)
u_in_half_mean_dict = train.groupby('breath_id')['u_in_half'].mean().to_dict()
train['u_in_half_mean'] = train['breath_id'].map(u_in_half_mean_dict)
u_in_half_std_dict = train.groupby('breath_id')['u_in_half'].std().to_dict()
train['u_in_half_std'] = train['breath_id'].map(u_in_half_std_dict)

In [ ]:
# Groupby is slow, do not use it.
# All entries are first point of each breath_id
first_df = train.loc[0::80,:]
# All entries are first point of each breath_id
last_df = train.loc[79::80,:]

In [ ]:
%%time
# The Main mode DataFrame and flag
main_df= last_df[(last_df['u_in']>4.8)&(last_df['u_in']<5.1)]
main_mode_dict = dict(zip(main_df['breath_id'], [1]*len(main_df)))
train['main_mode'] = train['breath_id'].map(main_mode_dict)
train['main_mode'].fillna(0, inplace=True)

In [ ]:
%%time
# u_out1_timing flag and DataFrame: speed up
# 高速版 uout1_df 作成
train['u_out_diff'] = train['u_out'].diff()
train['u_out_diff'].fillna(0, inplace=True)
train['u_out_diff'].replace(-1, 0, inplace=True)
uout1_df = train[train['u_out_diff']==1]

In [ ]:
main_uout1 = uout1_df[uout1_df['main_mode']==1]
nomain_uout1 = uout1_df[uout1_df['main_mode']==1]

In [ ]:
# Register Area when u_out becomes 1
uout1_area_dict = dict(zip(first_df['breath_id'], first_df['u_in']))
train['area_uout1'] = train['breath_id'].map(uout1_area_dict) 

In [ ]:
%%time
# u_in: first point, last point
u_in_first_dict = dict(zip(first_df['breath_id'], first_df['u_in']))
train['u_in_first'] = train['breath_id'].map(u_in_first_dict)
u_in_last_dict = dict(zip(first_df['breath_id'], last_df['u_in']))
train['u_in_last'] = train['breath_id'].map(u_in_last_dict)
# time(sec) of end point
time_end_dict = dict(zip(last_df['breath_id'], last_df['time_step']))     
train['time_end'] = train['breath_id'].map(time_end_dict)

In [ ]:
%%time
# time(sec) when u_out becomes 1
uout1_dict = dict(zip(uout1_df['breath_id'], uout1_df['time_step']))
train['time_uout1'] = train['breath_id'].map(uout1_dict)

In [ ]:
%%time
# u_in when u_out becomes1
u_in_uout1_dict = dict(zip(uout1_df['breath_id'], uout1_df['u_in']))
train['u_in_uout1'] = train['breath_id'].map(u_in_uout1_dict)

In [ ]:
%%time
# Dict that puts 0 at the beginning of the 80row cycle
first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

# Faster version u_in_diff creation, faster than groupby
train['u_in_diff'] = train['u_in'].diff()
train['tmp'] = train['id'].map(first_0_dict) # put 0, the 80row cycle
train.iloc[0::80, train.columns.get_loc('u_in_diff')] = train.iloc[0::80, train.columns.get_loc('tmp')]

In [ ]:
%%time
# Create u_in vibration
train['diff_sign'] = np.sign(train['u_in_diff'])
train['sign_diff'] = train['diff_sign'].diff()
train['tmp'] = train['id'].map(first_0_dict) # put 0, the 80row cycle
train.iloc[0::80, train.columns.get_loc('sign_diff')] = train.iloc[0::80, train.columns.get_loc('tmp')]

# Count the number of inversions, so take the absolute value and sum
train['sign_diff'] = abs(train['sign_diff']) 
sign_diff_dict = train.groupby('breath_id')['sign_diff'].sum().to_dict()
train['diff_vib'] = train['breath_id'].map(sign_diff_dict)

In [ ]:
%%time
if 'diff_sign' in train.columns:
    train.drop(['diff_sign', 'sign_diff'], axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
train.columns

# Recreate each DataFrame when all the features are available

In [ ]:
%%time
################################################################
first_df = train.loc[0::80,:]
last_df = train.loc[79::80,:]
main_df= last_df[(last_df['u_in']>4.8)&(last_df['u_in']<5.1)]
nomain_df = last_df[(last_df['u_in']<=4.8)|(last_df['u_in']>=5.1)]
uout1_df = train[train['u_out_diff']==1]
main_uout1 = uout1_df[uout1_df['main_mode']==1]
nomain_uout1 = uout1_df[uout1_df['main_mode']==1]
################################################################

# Functions for plot

In [ ]:
def plot_bid(bid, col1='', col2=''):
    fig, ax1 = plt.subplots(figsize = (6, 4)) 
    
    tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
    ax2 = ax1.twinx()

    ax1.plot(tmp['time_step'], tmp['pressure'], 'm-', label='pressure')
    ax1.plot(tmp['time_step'], tmp['u_in'], 'g-', label='u_in')
    ax2.plot(tmp['time_step'], tmp['u_out'], 'b-', label='u_out')

    ax1.set_xlabel('Timestep')
    
    R = tmp['R'][0]
    C = tmp['C'][0]
    mean = tmp['diff_mean'][0]
    std = tmp['diff_std'][0]
    vib = tmp['diff_vib'][0]
    title_str = f'breath_id:{bid}, R:{R}, C:{C}, mean:{mean:.2f}, std:{std:.2f}, vib:{vib:.1f}'
    if col1 != '':
        c1 = tmp[col1][0]
        title_str += f'{col1}: {c1}'
    if col2 != '':
        c2 = tmp[col2][0]
        title_str += f'{col2}: {c2}'
    ax1.set_title(title_str)

    ax1.set_ylim(0, 100)
    
    ax1.legend(loc=(1.1, 0.8))
    ax2.legend(loc=(1.1, 0.7))
    plt.show()

def plot_uin(bid):
    fig, ax1 = plt.subplots(figsize = (6, 4)) 

    tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
    #ax2 = ax1.twinx()

    ax1.plot(tmp['time_step'], tmp['u_in'], 'g-', label='u_in')

    ax1.set_xlabel('Timestep')
    
    R = tmp['R'][0]
    C = tmp['C'][0]
    mean = tmp['diff_mean'][0]
    std = tmp['diff_std'][0]
    vib = tmp['diff_vib'][0]
    title_str = f'breath_id:{bid}, R:{R}, C:{C}, mean:{mean:.2f}, std:{std:.2f}, vib:{vib:.1f}'
    ax1.set_title(title_str)

    ax1.set_ylim(0, 100)
    
    plt.show()

def plot_time_step(bid):
    plt.figure()
    tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
    R = tmp['R'][0]
    C = tmp['C'][0]
    plt.title(f'breath_id:{bid}, R:{R}, C:{C}')
    plt.ylabel('Timestep')
    plt.xlabel('Row No.')

    plt.plot(train.loc[train['breath_id'] == bid]['time_step'].tolist())
    plt.show()

def plot_uin_list(bid_list, ylim=100, u_low=0, u_high=100, pos=79, alpha=False):
    
    fig, ax1 = plt.subplots(figsize = (6, 4)) # original (12, 8)

    if alpha:
        a = alpha
    else:
        if (len(bid_list)):
            a = max(1.0/len(bid_list), 0.01)
        else:
            a = 1
    
    for bid in tqdm(bid_list):
        tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
        u = tmp['u_in'][pos]
        if (u >= u_low) and (u <= u_high):  
            ax1.plot(tmp['time_step'], tmp['u_in'], 'g-', alpha=a)

    ax1.set_xlabel('Timestep')
    ax1.set_ylabel('u_in')
    ax1.set_ylim(0,ylim)
    #ax1.legend(loc=(1.1, 0.8))
    plt.show()

def df_from_to(df, f=0.0, t=6.0):
    # 「0.0のみ」と「0.0より大きく1以下」を実現したいので、この不等号の形
    bid_list = df.loc[(df['u_in'] > f)&(df['u_in'] <= t)]['breath_id'].tolist()
    return bid_list   

In [ ]:
def plot_double_time_bid(bid, time_delta=False, col1='', col2=''):
    fig = plt.figure(figsize = (12, 4))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    
    tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)

    ts = []
    td = []
    if time_delta:
        outlier = tmp.loc[tmp['time_delta'] > 0.15]
        
        rw = outlier['id'].tolist()
        ts = outlier['time_step'].tolist()
        td = outlier['time_delta'].tolist()
        
    
    R = tmp['R'][0]
    C = tmp['C'][0]
    title_str = f'breath_id:{bid}, R:{R}, C:{C}'
    if col1 != '':
        c1 = tmp[col1][0]
        title_str += f'{col1}: {c1}'
    if col2 != '':
        c2 = tmp[col2][0]
        title_str += f'{col2}: {c2}'
    ax1.set_title(title_str)
    
    ax1.set_ylabel('Timestep')
    ax1.set_xlabel('Row No.')

    ymax = 3.0
    ax1.set_ylim(0, ymax)

    if time_delta:
        rows = []
        for a in rw:
            aa = a % 80 - 2
            if aa < 0:
                aa += 80
            rows.append(aa)
            aa = a % 80 - 1
            if aa < 0:
                aa += 80
            rows.append(aa)
        ax1.vlines(rows, 0, ymax, "red", linestyles='dashed', alpha=0.2)

    
    ax1.plot(train.loc[train['breath_id'] == bid]['time_step'].tolist())

    ##############################
    ax3 = ax2.twinx()

    ax2.plot(tmp['time_step'], tmp['pressure'], 'm-', label='pressure')
    ax2.plot(tmp['time_step'], tmp['u_in'], 'g-', label='u_in')
    ax3.plot(tmp['time_step'], tmp['u_out'], 'b-', label='u_out')

    ax2.set_xlabel('Timestep')
    
    R = tmp['R'][0]
    C = tmp['C'][0]
    mean = tmp['diff_mean'][0]
    std = tmp['diff_std'][0]
    vib = tmp['diff_vib'][0]
    title_str = f'breath_id:{bid}, R:{R}, C:{C}, mean:{mean:.2f}, std:{std:.2f}, vib:{vib:.1f}'
    ax2.set_title(title_str)

    ymax = 100
    ax2.set_ylim(0, ymax)
    
    if time_delta:
        lines = []
        for a, b in zip(ts, td):
            lines.append(a-b)
            lines.append(a)
        ax2.vlines(lines, 0, ymax, "red", linestyles='dashed', alpha=0.2)
    
    ax2.legend(loc=(1.1, 0.8))
    ax3.legend(loc=(1.1, 0.7))
    
    fig.tight_layout()
    plt.show()

In [ ]:
def plot_pre_list(bid_list, ylim=100, low=0, high=100, pos=79, alpha=False):
    
    fig, ax1 = plt.subplots(figsize = (6, 4)) # original (12, 8)

    if alpha:
        a = alpha
    else:
        if (len(bid_list)):
            a = max(1.0/len(bid_list), 0.01)
        else:
            a = 1
    
    for bid in tqdm(bid_list):
        tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
        u = tmp['pressure'][pos]
        if (u >= low) and (u <= high):  
            ax1.plot(tmp['time_step'], tmp['pressure'], 'm-', alpha=a)

    ax1.set_xlabel('Timestep')
    ax1.set_ylabel('Pressure')
    ax1.set_ylim(0,ylim)
    #ax1.legend(loc=(1.1, 0.8))
    plt.show()


In [ ]:
def plot_double_pre_list(bid_list, max_plots=False, ylim=100, alpha=False):
    fig = plt.figure(figsize = (12, 4))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
       
    title_str = f'time - u_in'
    ax1.set_title(title_str)
    
    ax1.set_ylabel('u_in')
    ax1.set_xlabel('Timestep')

    ax1.set_ylim(0, ylim)
   
    ##############################
    ax2.set_ylabel('Pressure')
    ax2.set_xlabel('Timestep')
    
    title_str = f'time - pressure'
    ax2.set_title(title_str)

    ax2.set_ylim(0, ylim)

    ##############################
    if alpha:
        a = alpha
    else:
        if (len(bid_list)):
            a = max(1.0/len(bid_list), 0.01)
        else:
            a = 1
    if not max_plots:
        max_plots = len(bid_list)
        
    for bid in tqdm(bid_list[:max_plots]):
        tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
        ax1.plot(tmp['time_step'], tmp['u_in'], 'g-', label='u_in', alpha=a)
        ax2.plot(tmp['time_step'], tmp['pressure'], 'm-', label='pressure', alpha=a)
    
    fig.tight_layout()
    plt.show()

In [ ]:
def plot_double_pre_list(bid_list, max_plots=False, ylim=100, alpha=False):
    fig = plt.figure(figsize = (12, 4))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
       
    title_str = f'time - u_in'
    ax1.set_title(title_str)
    
    ax1.set_ylabel('u_in')
    ax1.set_xlabel('Timestep')

    ax1.set_ylim(0, ylim)
   
    ##############################
    ax2.set_ylabel('Pressure')
    ax2.set_xlabel('Timestep')
    
    title_str = f'time - pressure'
    ax2.set_title(title_str)

    ax2.set_ylim(0, ylim)

    ##############################
    if alpha:
        a = alpha
    else:
        if (len(bid_list)):
            a = max(1.0/len(bid_list), 0.01)
        else:
            a = 1
    if not max_plots:
        max_plots = len(bid_list)
        
    for bid in tqdm(bid_list[:max_plots]):
        tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
        ax1.plot(tmp['time_step'], tmp['u_in'], 'g-', label='u_in', alpha=a)
        ax2.plot(tmp['time_step'], tmp['pressure'], 'm-', label='pressure', alpha=a)
    
    fig.tight_layout()
    plt.show()

In [ ]:
def plot_double_area_bid(bid):
    fig = plt.figure(figsize = (12, 4))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    
    tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)

    R = tmp['R'][0]
    C = tmp['C'][0]
    title_str = f'Area'
    ax1.set_title(title_str)
    
    ax1.set_ylabel('Area')
    ax1.set_xlabel('Timestep')

    ymax = 100
    ax1.set_ylim(0, ymax)

    ax1.plot(tmp['time_step'], tmp['area'],  'r-', label='area')
    #ax1.plot(tmp['time_step'], tmp['area2'], 'g-', label='area2')

    ##############################
    ax3 = ax2.twinx()

    ax2.plot(tmp['time_step'], tmp['pressure'], 'm-', label='pressure')
    ax2.plot(tmp['time_step'], tmp['u_in'], 'g-', label='u_in')
    ax3.plot(tmp['time_step'], tmp['u_out'], 'b-', label='u_out')

    ax2.set_xlabel('Timestep')
    
    R = tmp['R'][0]
    C = tmp['C'][0]
    mean = tmp['diff_mean'][0]
    std = tmp['diff_std'][0]
    vib = tmp['diff_vib'][0]
    title_str = f'breath_id:{bid}, R:{R}, C:{C}, mean:{mean:.2f}, std:{std:.2f}, vib:{vib:.1f}'
    ax2.set_title(title_str)

    ymax = 100
    ax2.set_ylim(0, ymax)
    
    ax2.legend(loc=(1.1, 0.8))
    ax3.legend(loc=(1.1, 0.7))
    
    fig.tight_layout()
    plt.show()

In [ ]:
def plot_bid_stats(list, indiv=3, df=last_df, max_plots=False, no_uin=False,
                   alpha=False, time_delta=False, col1='', col2=''):
    tmpdf= df[df['breath_id'].isin(list)]
    bid_list = tmpdf['breath_id']
    print('Number of plots:', len(bid_list))
    
    if indiv:
        for bid in bid_list[:indiv]: # 最大3個、個別表示
            if time_delta:
                plot_double_bid(bid, time_delta=True, col1=col1, col2=col2)
            else:
                plot_bid(bid, col1=col1, col2=col2)

    #plt.hist(tmpdf['R_C'], bins=17) # 棒グラフの順番指定できないので見にくい
    sns.countplot(x="R_C", data=tmpdf, order=RCorder)
    plt.show()
    
    if not no_uin:
        if not max_plots:
            max_plots = len(bid_list)
            print(f'Number of plots: {max_plots}')
        else:
            print(f'Number of plots: {max_plots}/{len(bid_list)}')
        plot_uin_list(bid_list[:max_plots], alpha=alpha)

# The Main mode and others

According to [notebook of mine](https://www.kaggle.com/marutama/eda-about-u-in), u_in has a main mode that accounts for 70018/75450=92%. It is 0 for 1 to 1.5 seconds and ends near 5 (4.965-4.995).

**Shows an overview of the two modes. Both u_in and pressue have distinctly different charts.**

In [ ]:
print('The main mode:')
plot_bid_stats(main_df['breath_id'], indiv=0, no_uin=True)
plot_double_pre_list(main_df['breath_id'][:1000])
print('Other than the main mode:')
plot_bid_stats(nomain_df['breath_id'], indiv=0, no_uin=True)
plot_double_pre_list(nomain_df['breath_id'][:1000])

In "other than main mode", there are only R = 50 and C = 10.

# Distribution of diff_vib: "u_in" diff vibration (Number of sign inversions)

**"diff_vib" also helps classify modes.**

In [ ]:
plt.hist(last_df['diff_vib'], bins=100)
plt.title("diff_vib global distribution")
plt.show()

In [ ]:
# The main mode and ohters
fig = plt.figure(figsize = (12, 4))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

ax1.hist(main_df['diff_vib'], bins=100)
ax1.set_title('The main mode')

ax2.hist(nomain_df['diff_vib'], bins=100)
ax2.set_title('Other than the main mode')

plt.show()

# EDA of other than the main mode

**From here, we will focus on "other than main mode". I don't know why, but R_C is only 50_10.**

In [ ]:
plot_bid_stats(nomain_df['breath_id'], indiv=False, max_plots=1000)

**Looking at the distribution of the "u_in" chart, it seems that it can be classified into two types: a chart that vibrates greatly and a chart that exists only below 15. Let's zoom in.**

In [ ]:
plot_double_pre_list(nomain_df['breath_id'][:1000], ylim=40)

Layered patterns can be seen in both the u_in graph and the pressure graph.
**Let's zoom in on the lower right corner of "u_in".**

In [ ]:
def close_up_bid_list(bid_list, xmin=0, xmax=2.8, ymin=0, ymax=100):
    fig, ax1 = plt.subplots(figsize = (6, 4)) # original (12, 8)

    for bid in tqdm(bid_list):
        tmp = train.loc[train['breath_id'] == bid].reset_index(drop=True)
        ax1.plot(tmp['time_step'], tmp['u_in'], 'g-', alpha=0.01)

    ax1.set_xlabel('Timestep')
    ax1.set_ylabel('u_in')
    ax1.set_xlim(xmin, xmax)
    ax1.set_ylim(ymin, ymax)
    plt.show()

In [ ]:
close_up_bid_list(nomain_df['breath_id'][:1000], ymin=0, ymax=7, xmin=2.55, xmax=2.7)

**It seems to be divided into a triangular wave chart that ends around 2.64 seconds and a straight chart that ends around 2.67 seconds. Shows the histogram.**

In [ ]:
ymax=10
plt.hist(nomain_df['time_end'], bins=100)
plt.ylim(0,ymax)
plt.vlines([2.65], 0, ymax, "red", linestyles='dashed')
plt.show()

**Let's classify by whether'time_end'is greater than 2.65 seconds.**

In [ ]:
df=nomain_df
nomain_u265=df[df['time_end']>2.65]  # upper 2.65
nomain_l265=df[df['time_end']<=2.65] # lower 2.65

In [ ]:
df=nomain_u265
print('No main mode, time_end > 2.65:', len(df))
plot_double_pre_list(df['breath_id'][:1000], ylim=40)

df=nomain_l265
print('No main mode, time_end <= 2.65:', len(df))
plot_double_pre_list(df['breath_id'][:1000])

**Interestingly, it seems that we were able to separate clearly different charts. I don't know why ...**

# "u_in" diff vibration (Number of sign inversions)

**You can clearly see the difference between time_end 2.65 and above and the following modes even with "diff_vib"..**

In [ ]:
plt.hist(nomain_u265['diff_vib'], bins=10, label='time end > 2.65', alpha=0.5)
plt.hist(nomain_l265['diff_vib'], bins=100, label='time end =< 2.65', alpha=0.5)
plt.title('Other than the Main mode: uin diff vibration')
plt.legend()
plt.show()

The vibration of "u_in diff" in "other than main mode" seems to have two modes. It is divided into time_end> 2.65 and time_end <= 2.65.

# Other than the main mode, time_end > 2.65

**This is the overview of "time_end> 2.65". This is the overview of "time_end> 2.65". The 6 layered pattern on the chart is impressive.**

In [ ]:
df=nomain_u265
print('No main mode, time_end > 2.65:', len(df))
plot_double_pre_list(df['breath_id'][:1000], ylim=40)

**If you look closely, there are two vertical lines around 1 second of pressure. Looking at the histogram of the time when u_out becomes 1, it is clearly divided into two.**

In [ ]:
df=nomain_u265
plt.hist(df['time_uout1'])
plt.show()

**However, it seems that the shape of the chart does not change much even with this. I'll leave it here.**

In [ ]:
df=nomain_u265
df2=df[df['time_uout1']<=1]
print('No main mode, time_end > 2.65 & time_uout1 <= 1:', len(df2))
plot_double_pre_list(df2['breath_id'][:1000], ylim=40)

In [ ]:
df=nomain_u265
df2=df[df['time_uout1'] > 1]
print('No main mode, time_end > 2.65 & time_uout1 > 1:', len(df2))
plot_double_pre_list(df2['breath_id'][:1000], ylim=40)

**Let's take a closer look at "time_end> 2.65". If you look at the histogram distribution of "u_in_mean", you can see 6 mountains.**

In [ ]:
ymax=18
plt.figure(figsize=(12,4))
plt.hist(nomain_u265['u_in_mean'], bins=500)
lines = [1, 1.9, 2.7, 3.6, 4.6, 5.7]
plt.ylim(0,ymax)
plt.vlines(lines, 0, ymax, "red", linestyles='dashed')
plt.title('Other than main mode, time_end > 2.65, histgram of u_in_mean')
plt.show()

**"time_end> 2.65" is the mode that can be most clearly classified by "u_in_mean".**

In [ ]:
df=nomain_u265
df2=df[(df['u_in_mean']>=1)&(df['u_in_mean']<1.9)]
plot_double_pre_list(df2['breath_id'], ylim=40)

In [ ]:
df=nomain_u265
df2=df[(df['u_in_mean']>=1.9)&(df['u_in_mean']<2.7)]
plot_double_pre_list(df2['breath_id'], ylim=40)

In [ ]:
df=nomain_u265
df2=df[(df['u_in_mean']>=2.7)&(df['u_in_mean']<3.6)]
plot_double_pre_list(df2['breath_id'], ylim=40)

In [ ]:
df=nomain_u265
df2=df[(df['u_in_mean']>=3.6)&(df['u_in_mean']<4.6)]
plot_double_pre_list(df2['breath_id'], ylim=40)

In [ ]:
df=nomain_u265
df2=df[(df['u_in_mean']>=4.6)&(df['u_in_mean']<5.7)]
plot_double_pre_list(df2['breath_id'], ylim=40)

In [ ]:
df=nomain_u265
df2=df[(df['u_in_mean']>=5.7)&(df['u_in_mean']<8.0)]
plot_double_pre_list(df2['breath_id'], ylim=40)

# Other than the main mode, time_end <= 2.65

**This is an overview of "time_end <= 2.65". Pressure has a clearer layered pattern than u_in.**

In [ ]:
df=nomain_l265
print(len(nomain_l265))
plot_double_pre_list(df['breath_id'][:1000])

When u_out1 becomes 1,'u_in' is divided into 0 and other 2 modes.

In [ ]:
plt.hist(nomain_l265['u_in_uout1'], bins=100)
plt.title('Other than main mode, time_end <= 2.65, histgram of u_in at u_out = 1')
plt.show()

**It is divided into 2000 or more 0s and others.**

In [ ]:
# out1 での uin が1か0かで分離
nomain_l265_u1u0 = nomain_l265[nomain_l265['u_in_uout1']==0]
nomain_l265_u1un = nomain_l265[nomain_l265['u_in_uout1']!=0]

Vibration also seems to have a mode

## Other than main mode, time_end < 2.65s, u_in==0 at u_out 1

**This is overview.**

In [ ]:
df=nomain_l265_u1u0 
print(len(df))
plot_double_pre_list(df['breath_id'][:1000])

**There are many vibrating charts, so take a look at the histogram of diff_vib.**

In [ ]:
plt.hist(nomain_l265_u1u0['diff_vib'], bins=100)
plt.vlines([80], 0, 110, "red", linestyles='dashed')
plt.title('histgram of diff_vib')
plt.show()

**Let's divide it into two by 80.**

### diff_vib <= 80 or diff_vib > 80

In [ ]:
df=nomain_l265_u1u0
nomain_l265_u1u0vl80 = df[df['diff_vib']<=80]
plot_double_pre_list(nomain_l265_u1u0vl80['breath_id'])

In [ ]:
df=nomain_l265_u1u0vl80
plt.figure(figsize=(12,4))
plt.title(f'u_in_half_mean: Non main mode, time_end <= 2.65, u_in==0@uout1, vib<=80')
plt.hist(df['u_in_mean'], bins=100)
plt.show()

In [ ]:
df=nomain_l265_u1u0
nomain_l265_u1u0vu80 = df[df['diff_vib']>80]
plot_double_pre_list(nomain_l265_u1u0vu80['breath_id'])

In [ ]:
df=nomain_l265_u1u0vu80
plt.figure(figsize=(12,4))
plt.title(f'u_in_half_mean: Non main mode, time_end <= 2.65, u_in==0@uout1, vib>80')
plt.hist(df['u_in_half_mean'], bins=100)
plt.show()

u_in_half_mean looks better than u_in_mean.

In [ ]:
df=nomain_l265_u1u0
nomain_l265_u1u0vu80 = df[df['diff_vib']>80]
plot_double_pre_list(nomain_l265_u1u0vu80['breath_id'])

In [ ]:
df=nomain_l265_u1u0vu80
plt.figure(figsize=(12,4))
plt.title(f'u_in_half_mean: Non main mode, time_end <= 2.65, u_in==0@uout1, vib>80')
plt.hist(df['u_in_half_mean'], bins=100)
plt.show()

This also seems to be better for u_in_half_mean than u_in_mean

## Other than main mode, time_end < 2.65s, u_in!=0 in out1

**This is overview.**

In [ ]:
df=nomain_l265_u1un 
print(len(df))
plot_double_pre_list(df['breath_id'][:1000])

**It seems to be divided into a gentle chart with u_in_max of 30 or less and a chart with more vibration. Let's take a look at the histogram of u_in_max.**

In [ ]:
df=nomain_l265_u1un
plt.hist(df['u_in_max'], bins=100)
plt.vlines([30], 0, 110, "red", linestyles='dashed')
plt.show()

**Let's divide u_in_max by 30.**

### u_in_max > 30

In [ ]:
df=nomain_l265_u1un
df = df[df['u_in_max']>30]
plot_double_pre_list(df['breath_id'])

In [ ]:
plt.figure(figsize=(12,4))
plt.title(f'u_in_half_mean: Non main mode, time_end <= 2.65, u_in!=0@uout1, u_in_max > 30: histgram of u_in_half_mean')
plt.hist(df['u_in_half_mean'], bins=100)
plt.show()

### u_in_max <= 30

In [ ]:
df=nomain_l265_u1un
df = df[df['u_in_max']<=30]
plot_double_pre_list(df['breath_id'])

In [ ]:
plt.figure(figsize=(12,4))
plt.title(f'u_in_half_mean: Non main mode, time_end <= 2.65, u_in!=0@uout1, uin_max > 30: histgram of u_in_half_mean')
plt.hist(df['u_in_half_mean'], bins=100)
plt.show()

Well, it's still difficult. Let's separate it further.

Zoom in.

In [ ]:
df=nomain_l265_u1un
df = df[df['u_in_max']<=30]
close_up_bid_list(df['breath_id'], xmax=0.5, ymax=30)

**There seems to be a vibrating chart and others. Let's look at the histogram.**

In [ ]:
df=nomain_l265_u1un
df = df[df['u_in_max']<=30]
plt.hist(df['diff_vib'], bins=50)
plt.vlines([50], 0, 175, "red", linestyles='dashed')
plt.show()

No mode.

In [ ]:
df=nomain_l265_u1un
df = df[df['u_in_max']<=30]
df = df[df['diff_vib']<=50]
plot_double_pre_list(df['breath_id'], ylim=45)

In [ ]:
close_up_bid_list(df['breath_id'], xmax=0.5, ymax=30)

No more modes can be divided.

In [ ]:
df=nomain_l265_u1un
df = df[df['u_in_max']<=30]
df = df[df['diff_vib']>=50]
plot_double_pre_list(df['breath_id'], ylim=45)

In [ ]:
close_up_bid_list(df['breath_id'], xmax=0.5, ymax=30)

Is it somehow successful?